# Clustering Crypto

In [442]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [443]:
import json
import pprint

In [444]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
requests = requests.get(url)
content = requests.content
content_cleaned = content.decode('utf-8')
data = json.loads(content_cleaned)

In [445]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_frame = pd.DataFrame(data=data["Data"])

crypto_dataframe = crypto_frame.T
crypto_dataframe

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TIGER,925903,/coins/tiger/overview,/media/34835667/cointiger.png,1539079984,TIGER,TIGER,TigerCash,TigerCash (TIGER),The official TigerCash ticker is “TCH” and tra...,N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPT,938079,/coins/gpt/overview,/media/37621865/grace-period-token.png,1608029024,GPT,GPT,Grace Period Token,Grace Period Token (GPT),GPT (Grace Period Token) is a token used to de...,N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCH,931525,/coins/tch/overview,/media/35651225/thch.png,1564479662,TCH,TCH,Thorecash,Thorecash (TCH),Thorecash makes crypto-lending and investments...,N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YLD,938086,/coins/yld/overview,/media/37621868/yieldapp.png,1608113117,YLD,YLD,YIELD App,YIELD App (YLD),"Yield is a P2P, non-custodial lending dapp tha...",N/A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [446]:
# Alternatively, use the provided csv file:
file_path = ("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [447]:
#Just did a check for myself has no effect on anything
list(crypto_df.columns)

['CoinName',
 'Algorithm',
 'IsTrading',
 'ProofType',
 'TotalCoinsMined',
 'TotalCoinSupply']

In [448]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [449]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df["IsTrading"] == False].index, inplace=True)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [450]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.dropna(subset=["Algorithm"], inplace=True)

In [451]:
# Remove the "IsTrading" column
crypto_df.drop(["IsTrading"], axis=1, inplace=True)

In [452]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [453]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df["TotalCoinsMined"] == 0].index, inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [454]:
# Drop rows where there are 'N/A' text values
crypto_df[crypto_df != "N/A"].dropna(inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [455]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = crypto_df["CoinName"]
coin_df = coin_names.to_frame()
coin_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [456]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(["CoinName"], axis=1, inplace=True)

In [457]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [458]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])

In [459]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [460]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_data_pca = pca.fit_transform(X_scaled)

In [461]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(crypto_data_pca, columns=["PC 1","PC 2","PC 3"], index=crypto_df.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.329519,1.029075,-0.538698
404,-0.312852,1.029441,-0.539022
1337,2.305992,1.681828,-0.612511
BTC,-0.145330,-1.344632,0.109220
ETH,-0.156455,-2.030988,0.304595
LTC,-0.161348,-1.110432,-0.026912
DASH,-0.399261,1.154552,-0.514625
XMR,-0.138563,-2.283643,0.250009
ETC,-0.154897,-2.031067,0.304593
ZEC,-0.159792,-2.031784,0.389811


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [462]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters = i, random_state = 0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k" : k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

## 5 is our best K value since that is where the curve turns showing an elbow

In [463]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
pcs_df["Class"] = model.labels_
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([crypto_df,coin_df,pcs_df], axis=1)
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC 1,PC 2,PC 3,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.329519,1.029075,-0.538698,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.312852,1.029441,-0.539022,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.305992,1.681828,-0.612511,1
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.145330,-1.344632,0.109220,0
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.156455,-2.030988,0.304595,0
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,ZEPHYR,2.482125,0.641823,-0.235978,1
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,Gapcoin,-0.327564,1.028986,-0.538704,1
BDX,CryptoNight,PoW,9.802226e+08,1400222610,Beldex,0.323153,-2.337696,0.331240,0
ZEN,Equihash,PoW,7.296538e+06,21000000,Horizen,-0.159793,-2.031784,0.389811,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [464]:
def get_clusters(k,data):
    model = KMeans(n_clusters=5, random_state=0)
    model.fit(pcs_df)

    predictions = model.predict(pcs_df)
    pcs_df["Class"] = model.labels_

    return data

In [465]:
five_clusters = get_clusters(4, clustered_df)
five_clusters.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC 1,PC 2,PC 3,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.329519,1.029075,-0.538698,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.312852,1.029441,-0.539022,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.305992,1.681828,-0.612511,1
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.145330,-1.344632,0.109220,0
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.156455,-2.030988,0.304595,0


In [466]:
 # Plotting the 3D-Scatter with x="Annual Income", y="Spending Score (1-100)" and z="Age"
fig = px.scatter_3d(
    five_clusters,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [467]:
# Table with tradable cryptos
tradable_crypto_table = clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)
tradable_crypto_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [468]:
# Print the total number of tradable cryptocurrencies
print(f"The total number of tradable cryptocurrencies is: {len(tradable_crypto_table)}")

The total number of tradable cryptocurrencies is: 533


#### Scatter Plot with Tradable Cryptocurrencies

In [469]:
trade_currencies = clustered_df.drop(columns=["Algorithm","ProofType","CoinName","PC 1", "PC 2", "PC 3"])
trade_currencies

,TotalCoinsMined,TotalCoinSupply,Class
42,4.199995e+01,42,1
404,1.055185e+09,532000000,1
1337,2.927942e+10,314159265359,1
BTC,1.792718e+07,21000000,0
ETH,1.076842e+08,0,0
...,...,...,...
ZEPH,2.000000e+09,2000000000,1
GAP,1.493105e+07,250000000,1
BDX,9.802226e+08,1400222610,0
ZEN,7.296538e+06,21000000,0


In [470]:
scaler = MinMaxScaler()
normalize = ["TotalCoinsMined", "TotalCoinSupply","Class"]
x = clustered_df[normalize].values
x_scaled = scaler.fit_transform(x)
tradable_currencies = pd.DataFrame(x_scaled, columns=normalize, index=crypto_df.index)
tradable_currencies

,TotalCoinsMined,TotalCoinSupply,Class
42,0.005942,4.200000e-11,0.25
404,0.007002,5.320000e-04,0.25
1337,0.035342,3.141593e-01,0.25
BTC,0.005960,2.100000e-05,0.00
ETH,0.006050,0.000000e+00,0.00
...,...,...,...
ZEPH,0.007951,2.000000e-03,0.25
GAP,0.005957,2.500000e-04,0.25
BDX,0.006927,1.400223e-03,0.00
ZEN,0.005950,2.100000e-05,0.00


In [471]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
 # Plotting the clusters
tradable_currencies.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols="CoinName",
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)